In [4]:
import pandas as pd
import math
import numpy as np

In [5]:
import locale
locale.setlocale(locale.LC_ALL, '')

'en_US.UTF-8'

In [6]:
!pip install uszipcode
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=False)

In [7]:
all_zips = pd.read_csv('~/Downloads/us-zip-code-latitude-and-longitude.csv', sep=";")
ca_zips = all_zips[all_zips['State'] == "CA"]['Zip'].values

fl_zips = all_zips[all_zips['State'] == 'FL']['Zip'].values

Disaster Function

In [8]:
# seperate numbers by commas
def place_value(number): 
    return ("{:,}".format(number)) 

In [9]:
search.by_zipcode(80302).keys()

['zipcode_type',
 'major_city',
 'post_office_city',
 'common_city_list',
 'county',
 'state',
 'lat',
 'lng',
 'timezone',
 'radius_in_miles',
 'area_code_list',
 'population',
 'population_density',
 'land_area_in_sqmi',
 'water_area_in_sqmi',
 'housing_units',
 'occupied_housing_units',
 'median_home_value',
 'median_household_income',
 'bounds_west',
 'bounds_east',
 'bounds_north',
 'bounds_south',
 'zipcode',
 'polygon',
 'population_by_year',
 'population_by_age',
 'population_by_gender',
 'population_by_race',
 'head_of_household_by_age',
 'families_vs_singles',
 'households_with_kids',
 'children_by_age',
 'housing_type',
 'year_housing_was_built',
 'housing_occupancy',
 'vancancy_reason',
 'owner_occupied_home_values',
 'rental_properties_by_number_of_rooms',
 'monthly_rent_including_utilities_studio_apt',
 'monthly_rent_including_utilities_1_b',
 'monthly_rent_including_utilities_2_b',
 'monthly_rent_including_utilities_3plus_b',
 'employment_status',
 'average_household_inc

In [10]:
z_dict = search.by_zipcode(26164)
population = z_dict.population
population

8262

In [14]:
z_dict.housing_units

3833

In [15]:
z_dict.occupied_housing_units

3429

In [21]:
z_dict.population

8262

In [64]:
# disaster level converting from category of storm to wind speed (km/hr)

def hurricane_damage(zip_code, disaster_level):
    try: 
        level = {
            1: 119, 2: 154, 3: 178, 4:209, 5: 252
        }
    
    except:
        pass

    z_dict = search.by_zipcode(zip_code)
    population = z_dict.population
       
    damage = -976532 + 2.22*(level[disaster_level])**3 + 9.81e-10*(population)**3
    damage = np.round(damage, 2)
    
    
    print(f'{zip_code} occured ${place_value(damage)} damage from a category {disaster_level} hurricane.')

In [65]:
hurricane_damage(26164, 1)

26164 occured $2,765,074.23 damage from a category 1 hurricane.


In [24]:
9.81e-10*(8262)**3

553.254021354168

In [25]:
9.81e-10*(30000)**3

26487.0

In [26]:
# # disaster level converting from category of storm to wind speed (km/hr)

# def hurricane_damage2(zip_code, disaster_level):
#     try: 
#         level = {
#             1: 119, 2: 154, 3: 178, 4:209, 5: 252
#         }
    
#     except:
#         pass

#     z_dict = search.by_zipcode(zip_code)
#     population = z_dict.population
    
#     damage = -2042821 + 316.55*(level[disaster_level])**2 + 6.58*(population) + 9.27e-10(population)**3 
#     damage = np.round(damage, 2)
    
    
#     print(f'{zip_code} occured {place_value(damage)} damage from a category {disaster_level} hurricane.')

In [27]:
# hurricane_damage2(26164, 1)

In [87]:
# Small Home <= 1000 sqft
# Average Home 1001-4999 sqft
# Large Home => 5000 sqft
# https://www.fema.gov/media-library-data/1499290622913-0bcd74f47bf20aa94998a5a920837710/Flood_Loss_Estimations_2017.pdf

def flood_loss(zip_code, water_inches):
    
    # getting number of houses in zipcode. setting to n_houses
    z_dict = search.by_zipcode(zip_code)
    n_houses = z_dict.housing_units
    
    water_inches = round(water_inches)
    
    inches = [13,14,15,16,17,18,19,20,21,22,23,25,26,27,28,29,30,31,32,33,34,35,37,38,39,40,41,42,43,44,45,46,47]
    
    if water_inches in inches:
        print(f'Not data for flood of {water_inches} inches')
        return
    elif water_inches > 48:
        print("No data for flood over 48 inches")
        return
    
    
    
    avg_sq_ft = 999 # This is where the avg_sq_ft per zipcode would go. 
    
    if avg_sq_ft < 1000:
    
        small_home = {1: 9550, 2: 9620, 3: 9820, 4: 12730, 5: 12780,
                      6: 15300, 7: 15508, 8: 15717, 9: 15925, 10: 16133,
                      11: 16342, 12: 16550, 24: 19500, 36: 21100, 48: 23400} 
        damage = small_home[water_inches] * n_houses
    
    
    elif avg_sq_ft >= 1000 and avg_sq_ft < 5000:
    
        average_home = {1: 23635, 2: 23720, 3: 24370, 4: 31345, 5: 31425, 
                        6: 37260, 7: 37691, 8: 38122, 9:38553, 10: 38983,
                        11: 39414, 12: 39845, 24: 44325, 36: 47905, 48: 53355}
        damage = average_home[water_inches] * n_houses
    
    
    elif avg_sq_ft > 5000:
    
        large_home = {1: 47110, 2: 47220, 3: 48620, 4: 62370, 5: 62500, 6: 73860,
                      7: 74662 , 8: 75463, 9: 76265, 10: 77067, 11: 77868, 12: 78670,
                      24: 85700, 36: 92580, 48: 103280}
        damage = large_home[water_inches] * n_houses
        
    else:
        
        print("Square footage not found")
   
    
    print(f'{zip_code} occured ${place_value(damage)} in damage from {water_inches} inches of flooding inside homes.')

In [88]:
flood_loss(77096, 4.3)

77096 occured $196,436,630 in damage from 4 inches of flooding inside homes.


In [89]:
flood_loss(26164, 14)

Not data for flood of 14 inches
